## Load packages

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import tensorflow as tf
import time
from keras import layers
from keras.layers import *
from keras.optimizers import *
from keras.models import Model
from keras.utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt

%matplotlib inline

/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:472: FutureWa

In [2]:
# set number of cores to 16
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=24, 
                                                   inter_op_parallelism_threads=24)))

## Load dataset

In [3]:
# load genetic data
chr1kg = np.load('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X.npz')

# S are samples, G are genotypes, V are variants
[(i,chr1kg[i].shape) for i in chr1kg]

[('S', (5008,)), ('G', (5008, 57876, 4)), ('V', (57876, 4))]

In [4]:
S=chr1kg['S'].astype(str)
S[:5]

array(['HG00096_S1', 'HG00097_S1', 'HG00099_S1', 'HG00100_S1',
       'HG00101_S1'], dtype='<U10')

In [5]:
V=chr1kg['V'].astype(str)
V[:2,:]

array([['1', '723307', 'C', 'G'],
       ['1', '727841', 'G', 'A']], dtype='<U225')

In [ ]:
G=chr1kg['G'].astype(bool)
G[:1,:1,:]

In [ ]:
# free up memory
chr1kg=[]

## Load ancestry labels

In [ ]:
sample_info=pd.read_csv('/home/jsokol/Data/igsr_samples.tsv', sep="\t")
sample_info.tail(3)

In [ ]:
## REMOVE AMERICANS + ADMIXED POPS
pops_to_remove=['ASW','MXL','GIH','ITU','STU','CEU','PUR','PEL','CLM']
samples=sample_info[~sample_info['Population code'].isin(pops_to_remove)]
samples.shape

In [ ]:
S=[i for i in S if i[:-3] in samples['Sample name'].values]
S_pop=[str(samples.loc[samples['Sample name']==i[:-3], "Superpopulation code"].values[0]) for i in S]
Y=np.array([[pop for _ in range(G.shape[1])] for pop in S_pop])
Y.shape

In [ ]:
Y[np.array([1,345,3012]),:8]

In [ ]:
labels=samples["Superpopulation code"].dropna().unique().tolist()
labels=dict(zip(labels, range(len(labels))))
k=len(labels)
(k, labels)

In [ ]:
Y2=[]
for i in range(Y.shape[0]): # individuals
    Y2.append([])
    for j in range(Y.shape[1]): # sites
        Y2[-1].append(np.zeros(k))
        Y2[-1][-1][labels[Y[i,j]]]=1
Y=np.array(Y2)
Y2=[]
print(Y.shape)

## Test-train split

In [ ]:
train=pd.read_csv('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X_int.train.txt', header=None).iloc[:,0].values
dev=pd.read_csv('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X_int.dev.txt', header=None).iloc[:,0].values
test=pd.read_csv('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X_int.test.txt', header=None).iloc[:,0].values

In [ ]:
train_ix=np.array([i for i,x in enumerate(S) if x in train])
dev_ix=np.array([i for i,x in enumerate(S) if x in dev])
test_ix=np.array([i for i,x in enumerate(S) if x in test])
[train_ix.shape, dev_ix.shape, test_ix.shape]

In [ ]:
train_X=G[train_ix,:,:]
train_Y=Y[train_ix,:]
[train_X.shape, train_Y.shape]

In [ ]:
dev_X=G[dev_ix,:,:]
dev_Y=Y[dev_ix,:]
[dev_X.shape, dev_Y.shape]

In [ ]:
test_X=G[test_ix,:,:]
test_Y=Y[test_ix,:]
[test_X.shape, test_Y.shape]

## Augment training set with admixed individuals

In [ ]:
# take number of ancestors as Pois(2.86 * generation_time), over 1-10 (maxgen) generations
n_fake=10000
maxgen=6
n_splits=2+np.hstack([ss.poisson.rvs(2.86*gen, size=n_fake//maxgen) for gen in range(1,maxgen)])

# new individuals
new_X=[]
new_Y=[]
for j in n_splits:
    if j==0:
        ind=np.random.choice(np.arange(train_X.shape[0]), size=1)
        new_X.append(list(train_X[ind,:,:]))
        new_Y.append(list(train_Y[ind,:,:]))
    # sample breakpoints uniformly
    breaks=np.sort(V.shape[0] * ss.beta.rvs(a=1, b=1, size=j)).astype(int)
    # pick founders uniformly at random without replacement and stitch their labels together
    founds=np.random.choice(np.arange(train_X.shape[0]), size=j+1, replace=False)
    # assemble genome and labels
    new_x,new_y = [],[]
    new_x.append(train_X[founds[0],:breaks[0],:])
    new_y.append(train_Y[founds[0],:breaks[0],:])
    for i,found in enumerate(founds[1:-1]):
        new_x.append(train_X[found, breaks[i]:breaks[i+1],:])
        new_y.append(train_Y[found, breaks[i]:breaks[i+1],:])
    new_x.append(train_X[founds[-1], breaks[-1]:,:])
    new_y.append(train_Y[founds[-1], breaks[-1]:,:])
    new_X.append(np.vstack(new_x))
    new_Y.append(np.vstack(new_y))
train_X=np.vstack((train_X, new_X))
train_Y=np.vstack((train_Y, new_Y))
[train_X.shape, train_Y.shape]

In [ ]:
plt.figure(figsize=(12,4))
plt.imshow(np.dot(train_Y[-1000:,:], np.arange(k)), aspect='auto', cmap='jet')

## Build model

In [ ]:
def model2(input_shape):
    """
    input_shape: The height, width and channels as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """
    # ref: https://github.com/zhixuhao/unet
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=input_shape)

    # First convolutional block
    #conv0 = Conv1D(filters=input_shape[1], kernel_size=1, padding='same')(X_input)
    #drop1 = Dropout(0.33)(conv0)
    conv1 = Conv1D(filters=128//2, kernel_size=256//2, padding = 'same')(X_input)
    conv1 = Activation('relu')(conv1)
    bn1 = BatchNormalization(axis = -1)(conv1)

    # Second convolutional block with maxpool
    conv2 = Conv1D(filters=256//2, kernel_size=64//2, padding = 'same', name = 'conv1')(bn1)
    bn2 = BatchNormalization(axis = -1)(conv2)
    conv2 = Activation('relu')(bn2)
    pool2 = MaxPooling1D(2)(conv2)

    # Third convolutional block with maxpool
    conv3 = Conv1D(filters=64//2, kernel_size=16//2, padding = 'same')(pool2)
    bn3 = BatchNormalization(axis = -1)(conv3)
    conv3 = Activation('relu')(bn3)    
    pool3 = MaxPooling1D(2)(conv3)
        
    # Now you just go back up
    conv4 = Conv1D(filters=256//2, kernel_size=64//2, padding='same')(UpSampling1D(size = 2)(pool3))
    conv4 = Activation('relu')(conv4)
    merge4= conv4 # concatenate([conv4, pool2], axis=1)
    conv5 = Conv1D(filters=256//2, kernel_size=64//2, padding='same')(merge4)
    conv5 = Activation('relu')(conv5)

    conv6 = Conv1D(filters=128//2, kernel_size=256//2, padding = 'same')(UpSampling1D(size = 2)(conv5))
    bn6 = BatchNormalization(axis = -1)(conv6)
    conv6 = Activation('relu')(bn6)
    merge6= conv6 # concatenate([conv6, bn1], axis=1)
    conv7 = Conv1D(filters=128//2, kernel_size=256//2, padding = 'same')(merge6)
    conv7 = Activation('relu')(conv7)
    Yhat = Conv1D(k, kernel_size=1, activation = 'softmax')(conv7)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = Yhat, name='model2b')
    
    return model

## Create and compile the model 

In [ ]:
# create model
nc,nv=train_X.shape[:2]
keep=np.random.choice(np.arange(train_X.shape[0]), replace=False, size=nc)
model = model2(train_X[keep,:nv,:].shape[1:])
# compile model
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
# summarize model
model.summary()

## Train model

In [ ]:
history=model.fit(train_X[keep,:nv,:], train_Y[keep,:nv,:], epochs = 4, batch_size = 32)

## Evaluate model

In [ ]:
print('\n# Evaluate on dev set')
results = model.evaluate(dev_X[:,:nv,:], dev_Y[:,:nv,:], batch_size=1)
print('dev set loss, acc:', results)

In [ ]:
model.save('cnn_lai_v01_20200306.h5')

In [ ]:
_, train_acc = model.evaluate(train_X[keep,:nv,:], train_Y[keep,:nv,:], verbose=0)
_, dev_acc = model.evaluate(dev_X[:,:nv,:], dev_Y[:,:nv,:], verbose=0)

In [ ]:
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
print('train acc:', train_acc)
print('train acc:', test_acc)